#IMPORT

In [1]:
!pip install matplotlib
!pip install gdown
!pip install xgboost


[notice] A new release of pip is available: 24.3.1 -> 25.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.3.1 -> 25.1
[notice] To update, run: python.exe -m pip install --upgrade pip


   ---------------------------------------- 0.0/150.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/150.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/150.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/150.0 MB ? eta -:--:--
   ---------------------------------------- 0.3/150.0 MB ? eta -:--:--
   ---------------------------------------- 0.3/150.0 MB ? eta -:--:--
   ---------------------------------------- 0.3/150.0 MB ? eta -:--:--
   ---------------------------------------- 0.3/150.0 MB ? eta -:--:--
   ---------------------------------------- 0.5/150.0 MB 364.6 kB/s eta 0:06:50
   ---------------------------------------- 0.5/150.0 MB 364.6 kB/s eta 0:06:50
   ---------------------------------------- 0.5/150.0 MB 364.6 kB/s eta 0:06:50
   ---------------------------------------- 0.8/150.0 MB 353.1 kB/s eta 0:07:03
   ---------------------------------------- 0.8/150.0 MB 353.1 kB/s eta 0:07:03
   -----------------------------

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV

from xgboost import XGBRegressor

In [2]:
import pandas as pd
import numpy as np
import os
import gdown
import matplotlib as plt
import re
import glob

#DATA PREPROCESSING

In [10]:
urls = {
    "2020": "https://drive.google.com/uc?id=1ULI0h3EKo5kuUMsT2xFnUD8i202QJjYZ",
    "2021": "https://drive.google.com/uc?id=12M2eDIfMP6wJAnmziGmkl11lwraF68NZ",
    "2022": "https://drive.google.com/uc?id=1cldDmJ7Aknjqw7eC_8T9_kSf2f9ysniG",
    "2023": "https://drive.google.com/uc?id=1nff4fbOEV7O0rIMKV9fAIme26LID0IC_",
    "2024": "https://drive.google.com/uc?id=1-r_Grt20TvrmMEFBaLKjcTxzLBKTnnV7"
}

os.makedirs("./data", exist_ok=True)

for year, url in urls.items():
    output = f"./data/vnexpress_{year}_clearned.csv"
    print(f"Downloading {year}...")
    gdown.download(url, output, quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1ULI0h3EKo5kuUMsT2xFnUD8i202QJjYZ
To: d:\Grab\predict\data\vnexpress_2020_clearned.csv
100%|██████████| 301k/301k [00:00<00:00, 1.25MB/s]


Downloading...
From: https://drive.google.com/uc?id=12M2eDIfMP6wJAnmziGmkl11lwraF68NZ
To: d:\Grab\predict\data\vnexpress_2021_clearned.csv
100%|██████████| 344k/344k [00:00<00:00, 1.32MB/s]


Downloading...
From: https://drive.google.com/uc?id=1cldDmJ7Aknjqw7eC_8T9_kSf2f9ysniG
To: d:\Grab\predict\data\vnexpress_2022_clearned.csv
100%|██████████| 375k/375k [00:00<00:00, 1.36MB/s]


Downloading...
From: https://drive.google.com/uc?id=1nff4fbOEV7O0rIMKV9fAIme26LID0IC_
To: d:\Grab\predict\data\vnexpress_2023_clearned.csv
100%|██████████| 448k/448k [00:00<00:00, 1.35MB/s]


Downloading...
From: https://drive.google.com/uc?id=1-r_Grt20TvrmMEFBaLKjcTxzLBKTnnV7
To: d:\Grab\predict\data\vnexpress_2024_clearned.csv
100%|██████████| 622k/622k [00:00<00:00, 1.39MB/s]


In [ ]:
all_dfs = []

for file in glob.glob("./data/vnexpress_*_clearned.csv"):
    match = re.search(r"(\d{4})", file)
    if match:
        nam = int(match.group(1))  # Lấy năm tìm được
        print(f"Working with {file} ({nam})...")
        df = pd.read_csv(file)
        df["nam"] = nam
        all_dfs.append(df)
    else:
        print(f"Cannot find {file}")

if all_dfs:
    data = pd.concat(all_dfs, ignore_index=True)
    data = data.drop(['hoc_phi'], axis=1)
    
    print("All the data: ")
    print(data.head())
    
    # Lưu dữ liệu vào file data.csv
    data.to_csv('./data/data.csv', index=False)
    print("Done downloading")
else:
    print("There is nothing to work with")

Working with ./data\vnexpress_2020_clearned.csv (2020)...
Working with ./data\vnexpress_2021_clearned.csv (2021)...
Working with ./data\vnexpress_2022_clearned.csv (2022)...
Working with ./data\vnexpress_2023_clearned.csv (2023)...
Working with ./data\vnexpress_2024_clearned.csv (2024)...
All the data: 
  ma_truong                ten_truong dia_diem  ...   diem     to_hop_mon   nam
0       BKA  Đại học Bách khoa Hà Nội   Hà Nội  ...  26.50  A00, B00, D07  2020
1       BKA  Đại học Bách khoa Hà Nội   Hà Nội  ...  23.18  A00, A01, D07  2020
2       BKA  Đại học Bách khoa Hà Nội   Hà Nội  ...  27.15       A00, A01  2020
3       BKA  Đại học Bách khoa Hà Nội   Hà Nội  ...  25.03  A01, D01, D07  2020
4       BKA  Đại học Bách khoa Hà Nội   Hà Nội  ...  26.75       A00, A01  2020

[5 rows x 8 columns]
Done dowloading


In [5]:
csv_path = './data/data.csv'

df = pd.read_csv(csv_path)

nganh_lon_hon_50 = df[df['diem'] > 50]
print(f"Số ngành có điểm > 50: {len(nganh_lon_hon_50)}")
print("\nMột số ngành có điểm lớn hơn 50:")
print(nganh_lon_hon_50[['ma_truong', 'ma_nganh', 'ten_nganh', 'diem']].head())

df.loc[(df['ma_truong'] == 'NLS') & (df['ma_nganh'] == '7340101'), 'diem'] = 23.3
df.to_csv(csv_path, index=False)
print("Đã sửa điểm ngành NLS - 7340101 thành 23.3 và lưu lại.")

Số ngành có điểm > 50: 125

Một số ngành có điểm lớn hơn 50:
     ma_truong ma_nganh                                          ten_nganh  \
5990       QSB      141                              Bảo dưỡng Công nghiệp   
5991       QSB      268            Cơ Kỹ thuật - CLC Tăng cường Tiếng Nhật   
5992       QSB      138                                        Cơ Kỹ thuật   
5993       QSB      218                                 Công nghệ sinh học   
5994       QSB      219  Công nghệ Thực phẩm (CT Chất lượng cao, giảng ...   

       diem  
5990  59.51  
5991  62.37  
5992  63.17  
5993  63.99  
5994  63.22  
Đã sửa điểm ngành NLS - 7340101 thành 23.3 và lưu lại.


In [34]:
#Folder cho các thư mục dự đoán sau này
#Do data đang có các ngành theo thang điểm khác nhau: 30, 40, 90 (qsb)
folders = ['qsb', 'thang_30', 'thang_40']
for folder in folders:
    os.makedirs(folder, exist_ok=True)

print("Create all the folders for data")

Create all the folders for data


In [52]:
#Only for QSB
def Preprocess_QSB(data_file = './data/data.csv'):
    df = pd.read_csv(data_file)
    df_qsb = df[(df['ma_truong'] == 'QSB') & (df['nam'].isin([2022, 2023, 2024]))]

    # Khúc này xử lý các ngành mới có trong 2024 (idea)
    majors_2022_2023 = set(df_qsb[df_qsb['nam'].isin([2022, 2023])]['ma_nganh'])
    majors_2024 = set(df_qsb[df_qsb['nam'] == 2024]['ma_nganh'])
    new_major_2024 = majors_2024 - majors_2022_2023
    df_qsb['newmajor'] = df_qsb['ma_nganh'].apply(lambda x: 1 if x in new_major_2024 else 0)
    print("New majors that appeared in 2024 (newmajor = 1):")
    print(df_qsb[df_qsb['newmajor'] == 1][['ma_nganh', 'ten_nganh']].drop_duplicates())

    df_qsb.to_csv('qsb/qsb_only.csv', index=False)
    print("Filtered data saved to qsb/qsb_only.csv")

In [ ]:
def Preprocess(data_file = './data/data.csv', output='', thang30 = True):
    df = pd.read_csv(data_file)
    if thang30 == True:
        df_other = df[(df['ma_truong'] != 'QSB') & (df['diem'] < 30)]
        print("Data < 30")
    else:
        df_other = df[(df['ma_truong'] != 'QSB') & (df['diem'] >= 30)]
        print("Data >= 30")
    
    majors_2020_2023 = set(df_other[df_other['nam'].isin([2020, 2021, 2022, 2023])]['ma_nganh'])
    majors_2024 = set(df_other[df_other['nam'] == 2024]['ma_nganh'])
    new_major_2024 = majors_2024 - majors_2020_2023
    df_other['newmajor'] = df_other['ma_nganh'].apply(lambda x: 1 if x in new_major_2024 else 0)
    
    print("New majors that appeared in 2024 (newmajor = 1):")
    new_major_2024 = df_other[df_other['newmajor'] == 1]
    print(new_major_2024[['ma_nganh', 'ten_nganh']])
    
    if df_other.empty:
        print("No data found for non-QSB schools in the specified years.")
        return
    
    df_other.to_csv(output, index=False)
    print(f"Filtered data for non-QSB schools saved to {output}")

In [122]:
Preprocess_QSB(data_file = './data/data.csv')
Preprocess(data_file = './data/data.csv', output='./thang_30/thang_30.csv', thang30=True)
Preprocess(data_file = './data/data.csv', output='./thang_40/thang_40.csv', thang30=False)

New majors that appeared in 2024 (newmajor = 1):
      ma_nganh              ten_nganh
13203      147  Địa Kỹ thuật xây dựng
13205      146       Khoa học dữ liệu
13211      148       Kinh tế xây dựng
Filtered data saved to qsb/qsb_only.csv
Data < 30
New majors that appeared in 2024 (newmajor = 1):
            ma_nganh                                          ten_nganh
12732            MS5                                        Kỹ thuật in
12736            EM1                                 Quản lý năng lượng
12740            ED3                                   Quản lý giáo dục
12750          NTH10  Định hướng nghề nghiệp Khoa học máy tính và dữ...
12754          NTH06                                Ngôn ngữ Trung Quốc
...              ...                                                ...
17555  7860226|21A00  Chỉ huy Kỹ thuật Phòng không - Không quân (Nam...
17559        7310109                                         Kinh tế số
17562        7310109                                

C:\Users\Admin\AppData\Local\Temp\ipykernel_3868\3062718917.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_qsb['newmajor'] = df_qsb['ma_nganh'].apply(lambda x: 1 if x in new_major_2024 else 0)
C:\Users\Admin\AppData\Local\Temp\ipykernel_3868\206259075.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_other['newmajor'] = df_other['ma_nganh'].apply(lambda x: 1 if x in new_major_2024 else 0)
C:\Users\Admin\AppData\Local\Temp\ipykernel_3868\206259075.py:13: SettingWithCopyWarning: 
A value is try

In [69]:
def extract_subject_combinations(file_path, save_txt=True):
    df = pd.read_csv(file_path)

    def extract_combinations(to_hop_mon):
        if isinstance(to_hop_mon, str):
            return [combo.strip() for combo in to_hop_mon.split(',')]
        return []

    all_combinations = set()
    for to_hop_mon in df['to_hop_mon']:
        combinations = extract_combinations(to_hop_mon)
        all_combinations.update(combinations)

    unique_combinations = sorted(list(all_combinations))
    print("List of unique subject combinations:")
    for combo in unique_combinations:
        print(combo)

    if save_txt:
        txt_path = os.path.splitext(file_path)[0] + '.txt'
        os.makedirs(os.path.dirname(txt_path), exist_ok=True)
        with open(txt_path, 'w') as f:
            for combo in unique_combinations:
                f.write(combo + '\n')
        print(f"\nList saved to '{txt_path}'")

    return unique_combinations

In [70]:
extract_subject_combinations('qsb/qsb_only.csv')
extract_subject_combinations('thang_30/thang_30.csv')
extract_subject_combinations('thang_40/thang_40.csv')

List of unique subject combinations:
A00
A01
B00
B08
D01
D07

List saved to 'qsb/qsb_only.txt'
List of unique subject combinations:
A00
A01
A02
A03
A04
A05
A06
A07
A08
A09
A10
A11
A12
A14
A15
A16
A17
B00
B01
B02
B03
B04
B05
B08
C00
C01
C02
C03
C04
C05
C08
C09
C10
C13
C14
C15
C19
C20
D01
D02
D03
D04
D05
D06
D07
D08
D09
D10
D11
D12
D13
D14
D15
D19
D22
D23
D24
D25
D26
D28
D29
D33
D42
D43
D44
D45
D63
D64
D66
D71
D72
D78
D80
D81
D82
D83
D84
D90
D91
D96
D97
H00
H01
H02
H03
H04
H05
H06
H07
H08
K01
M00
M01
M02
M03
M04
M05
M06
M07
M08
M09
M10
M11
M13
M14
N00
N01
N03
N05
R01
R02
R03
R04
S00
T00
T01
T02
T03
T04
T05
T06
T07
V00
V01
V02
V03
V04
V05
V06
V07
V08
V09
V10

List saved to 'thang_30/thang_30.txt'
List of unique subject combinations:
A00
A01
A02
A06
A16
B00
B03
B08
C00
C01
C03
C04
C14
C15
C19
D01
D02
D03
D04
D05
D06
D07
D08
D09
D10
D11
D12
D14
D15
D55
D66
D72
D78
D84
D90
D96
H00
H01
H02
H03
M00
M01
M05
M09
N00
T00
T01
T02
T05
V00
V01
V02

List saved to 'thang_40/thang_40.txt'


['A00',
 'A01',
 'A02',
 'A06',
 'A16',
 'B00',
 'B03',
 'B08',
 'C00',
 'C01',
 'C03',
 'C04',
 'C14',
 'C15',
 'C19',
 'D01',
 'D02',
 'D03',
 'D04',
 'D05',
 'D06',
 'D07',
 'D08',
 'D09',
 'D10',
 'D11',
 'D12',
 'D14',
 'D15',
 'D55',
 'D66',
 'D72',
 'D78',
 'D84',
 'D90',
 'D96',
 'H00',
 'H01',
 'H02',
 'H03',
 'M00',
 'M01',
 'M05',
 'M09',
 'N00',
 'T00',
 'T01',
 'T02',
 'T05',
 'V00',
 'V01',
 'V02']

#TRAIN VÀ PREDICT

In [235]:
def predict_admission_scores(csv_file, combo_file, output_file, train_years):
    # ==== B1: Đọc dữ liệu ====
    with open(combo_file, 'r') as f:
        common_combinations = [line.strip() for line in f if line.strip()]
    print(" Danh sách tổ hợp môn:", common_combinations)

    df = pd.read_csv(csv_file)

    # ==== B2: Tiền xử lý dữ liệu ====
    le_nganh = LabelEncoder()
    df['ma_nganh_encoded'] = le_nganh.fit_transform(df['ma_nganh'])

    for combo in common_combinations:
        df[f'has_{combo}'] = df['to_hop_mon'].apply(lambda x: 1 if combo in x else 0)

    df['nam_truoc'] = df.groupby('ma_nganh')['nam'].shift(1)
    df['nam_truoc'].fillna(0, inplace=True) 
    df['diem_truoc'] = df.groupby('ma_nganh')['diem'].shift(1)
    df['diem_truoc'].fillna(0, inplace=True) 
    df['diem_trend'] = np.where(df['nam'] - df['nam_truoc'] == 1, df['diem'] - df['diem_truoc'], 0)

    # ==== B3: Huấn luyện model XGBoost ====
    train_df = df[df['nam'].isin(train_years)]

    features = ['nam', 'ma_nganh_encoded', 'diem_trend', 'newmajor'] + [f'has_{combo}' for combo in common_combinations]
    X_train = train_df[features]
    y_train = train_df['diem']

    xgb_model = XGBRegressor(objective='reg:squarederror', random_state=42)

    param_grid = {
        'n_estimators': [100, 200],
        'max_depth': [3, 6, 10],
        'learning_rate': [0.01, 0.1, 0.2],
        'subsample': [0.7, 1.0],
        'colsample_bytree': [0.7, 1.0]
    }

    grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid,
                               cv=5, scoring='neg_mean_squared_error', n_jobs=-1, verbose=2)
    grid_search.fit(X_train, y_train)
    best_xgb_model = grid_search.best_estimator_
    print("\n Tham số tốt nhất:", grid_search.best_params_)

    # ==== B4: Chuẩn bị test_df_2025 từ dữ liệu 2024 ====
    last_year = max(train_years)
    test_year = last_year + 1

    test_df = df[df['nam'] == last_year].copy()
    test_df['nam'] = test_year

    trend_mean = df.groupby('ma_nganh')['diem_trend'].mean().reset_index()
    trend_mean = trend_mean.rename(columns={'diem_trend': 'diem_trend_mean'})
    test_df = pd.merge(test_df, trend_mean, on='ma_nganh', how='left')

    test_df['diem_trend'] = np.where(
        test_df['newmajor'] == 0,
        test_df['diem_trend_mean'].fillna(0),
        0
    )

    test_df = test_df.drop(columns=['diem_trend_mean'])

    # ==== B5: Dự đoán điểm chuẩn năm test_year ====
    X_test = test_df[features]
    test_df['predicted_diem'] = np.round(best_xgb_model.predict(X_test), 2)
    
    # ==== B6: Xuất file kết quả ====
    output_df = test_df[['ma_truong', 'ten_truong', 'dia_diem', 'ma_nganh', 'ten_nganh',
                         'to_hop_mon', 'predicted_diem', 'newmajor']]
    output_df['nam'] = test_year
    output_df.to_csv(output_file, index=False)
    print(f" Đã lưu dự đoán năm {test_year} vào '{output_file}'.")

    # ==== B7: Kiểm tra nếu có dữ liệu thật cho test_year và tính metric ====
    if test_year in df['nam'].values:
        # Merge để đảm bảo true_diem có thể gán đúng
        true_values_df = df[df['nam'] == test_year][['ma_nganh', 'diem']]
        test_df = pd.merge(test_df, true_values_df, on='ma_nganh', how='left', suffixes=('', '_true'))

        # Lọc ra những ngành có dữ liệu thực (diem)
        valid_test_df = test_df.dropna(subset=['diem_true'])

        # Kiểm tra số lượng mẫu có khớp không
        if len(valid_test_df) > 0:
            mae = mean_absolute_error(valid_test_df['diem_true'], valid_test_df['predicted_diem'])
            mse = mean_squared_error(valid_test_df['diem_true'], valid_test_df['predicted_diem'])
            r2 = r2_score(valid_test_df['diem_true'], valid_test_df['predicted_diem'])

            print(f" MAE: {mae}")
            print(f" MSE: {mse}")
            print(f" R^2: {r2}")
        else:
            print(f" Không có đủ ngành hợp lệ để tính metric (ngành mới không có dữ liệu thật).")
    else:
        print(f" Không có dữ liệu thật cho năm {test_year}, không tính được các metric.")

    # ==== B7: In danh sách ngành mới ====
    print(f"\n Các ngành mới (newmajor == 1) trong năm {test_year} và điểm dự đoán:")
    print(output_df[output_df['newmajor'] == 1][['ma_nganh', 'ten_nganh', 'predicted_diem']].to_string(index=False))


#QSB

In [236]:
predict_admission_scores('qsb/qsb_only.csv', 'qsb/qsb_only.txt', 'qsb/2024_qsb.csv', train_years=[2022, 2023])

 Danh sách tổ hợp môn: ['A00', 'A01', 'B00', 'B08', 'D01', 'D07']
Fitting 5 folds for each of 72 candidates, totalling 360 fits


C:\Users\Admin\AppData\Local\Temp\ipykernel_3868\2512395822.py:17: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['nam_truoc'].fillna(0, inplace=True)
C:\Users\Admin\AppData\Local\Temp\ipykernel_3868\2512395822.py:19: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, whe


 Tham số tốt nhất: {'colsample_bytree': 0.7, 'learning_rate': 0.2, 'max_depth': 3, 'n_estimators': 200, 'subsample': 0.7}
 Đã lưu dự đoán năm 2024 vào 'qsb/2024_qsb.csv'.
 MAE: 7.083499874114989
 MSE: 72.20431800201663
 R^2: -0.1002464705964643

 Các ngành mới (newmajor == 1) trong năm 2024 và điểm dự đoán:
Empty DataFrame
Columns: [ma_nganh, ten_nganh, predicted_diem]
Index: []


C:\Users\Admin\AppData\Local\Temp\ipykernel_3868\2512395822.py:71: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  output_df['nam'] = test_year


In [237]:
predict_admission_scores('qsb/qsb_only.csv', 'qsb/qsb_only.txt', 'qsb/2025_qsb.csv', train_years=[2022, 2023, 2024])

 Danh sách tổ hợp môn: ['A00', 'A01', 'B00', 'B08', 'D01', 'D07']
Fitting 5 folds for each of 72 candidates, totalling 360 fits


C:\Users\Admin\AppData\Local\Temp\ipykernel_3868\2512395822.py:17: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['nam_truoc'].fillna(0, inplace=True)
C:\Users\Admin\AppData\Local\Temp\ipykernel_3868\2512395822.py:19: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, whe


 Tham số tốt nhất: {'colsample_bytree': 0.7, 'learning_rate': 0.1, 'max_depth': 10, 'n_estimators': 100, 'subsample': 0.7}
 Đã lưu dự đoán năm 2025 vào 'qsb/2025_qsb.csv'.
 Không có dữ liệu thật cho năm 2025, không tính được các metric.

 Các ngành mới (newmajor == 1) trong năm 2025 và điểm dự đoán:
 ma_nganh             ten_nganh  predicted_diem
      147 Địa Kỹ thuật xây dựng       55.500000
      146      Khoa học dữ liệu       79.910004
      148      Kinh tế xây dựng       59.560001


C:\Users\Admin\AppData\Local\Temp\ipykernel_3868\2512395822.py:71: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  output_df['nam'] = test_year


#THANG_40

In [238]:
predict_admission_scores('thang_40/thang_40.csv', 'thang_40/thang_40.txt', 'thang_40/2024_thang_40.csv', train_years=[2020, 2021, 2022, 2023])


 Danh sách tổ hợp môn: ['A00', 'A01', 'A02', 'A06', 'A16', 'B00', 'B03', 'B08', 'C00', 'C01', 'C03', 'C04', 'C14', 'C15', 'C19', 'D01', 'D02', 'D03', 'D04', 'D05', 'D06', 'D07', 'D08', 'D09', 'D10', 'D11', 'D12', 'D14', 'D15', 'D55', 'D66', 'D72', 'D78', 'D84', 'D90', 'D96', 'H00', 'H01', 'H02', 'H03', 'M00', 'M01', 'M05', 'M09', 'N00', 'T00', 'T01', 'T02', 'T05', 'V00', 'V01', 'V02']
Fitting 5 folds for each of 72 candidates, totalling 360 fits


C:\Users\Admin\AppData\Local\Temp\ipykernel_3868\2512395822.py:17: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['nam_truoc'].fillna(0, inplace=True)
C:\Users\Admin\AppData\Local\Temp\ipykernel_3868\2512395822.py:19: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, whe


 Tham số tốt nhất: {'colsample_bytree': 1.0, 'learning_rate': 0.1, 'max_depth': 10, 'n_estimators': 100, 'subsample': 1.0}
 Đã lưu dự đoán năm 2024 vào 'thang_40/2024_thang_40.csv'.
 MAE: 1.0560937778155013
 MSE: 1.9737781718900334
 R^2: 0.2681034372698805

 Các ngành mới (newmajor == 1) trong năm 2024 và điểm dự đoán:
Empty DataFrame
Columns: [ma_nganh, ten_nganh, predicted_diem]
Index: []


C:\Users\Admin\AppData\Local\Temp\ipykernel_3868\2512395822.py:71: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  output_df['nam'] = test_year


In [239]:
predict_admission_scores('thang_40/thang_40.csv', 'thang_40/thang_40.txt', 'thang_40/2025_thang_40.csv', train_years=[2020, 2021, 2022, 2023, 2024])


 Danh sách tổ hợp môn: ['A00', 'A01', 'A02', 'A06', 'A16', 'B00', 'B03', 'B08', 'C00', 'C01', 'C03', 'C04', 'C14', 'C15', 'C19', 'D01', 'D02', 'D03', 'D04', 'D05', 'D06', 'D07', 'D08', 'D09', 'D10', 'D11', 'D12', 'D14', 'D15', 'D55', 'D66', 'D72', 'D78', 'D84', 'D90', 'D96', 'H00', 'H01', 'H02', 'H03', 'M00', 'M01', 'M05', 'M09', 'N00', 'T00', 'T01', 'T02', 'T05', 'V00', 'V01', 'V02']
Fitting 5 folds for each of 72 candidates, totalling 360 fits


C:\Users\Admin\AppData\Local\Temp\ipykernel_3868\2512395822.py:17: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['nam_truoc'].fillna(0, inplace=True)
C:\Users\Admin\AppData\Local\Temp\ipykernel_3868\2512395822.py:19: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, whe


 Tham số tốt nhất: {'colsample_bytree': 0.7, 'learning_rate': 0.1, 'max_depth': 10, 'n_estimators': 200, 'subsample': 1.0}
 Đã lưu dự đoán năm 2025 vào 'thang_40/2025_thang_40.csv'.
 Không có dữ liệu thật cho năm 2025, không tính được các metric.

 Các ngành mới (newmajor == 1) trong năm 2025 và điểm dự đoán:
  ma_nganh                                                 ten_nganh  predicted_diem
   7480202                                         An toàn thông tin       35.070000
   7480104                                        Hệ thống thông tin       35.900002
      EP15                                          Khoa học dữ liệu       35.340000
      EP17                                         Kỹ thuật phần mềm       34.090000
      EP18                              Quản trị giải trí và sự kiện       36.540001
      EP16                                          Trí tuệ nhân tạo       34.490002
7340101C35                  Digital Marketing (theo định hướng ICDL)       35.299999
7340201C

C:\Users\Admin\AppData\Local\Temp\ipykernel_3868\2512395822.py:71: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  output_df['nam'] = test_year


#THANG_30

In [240]:
predict_admission_scores('thang_30/thang_30.csv', 'thang_30/thang_30.txt', 'thang_30/2024_thang_30.csv', train_years=[2020, 2021, 2022, 2023])


 Danh sách tổ hợp môn: ['A00', 'A01', 'A02', 'A03', 'A04', 'A05', 'A06', 'A07', 'A08', 'A09', 'A10', 'A11', 'A12', 'A14', 'A15', 'A16', 'A17', 'B00', 'B01', 'B02', 'B03', 'B04', 'B05', 'B08', 'C00', 'C01', 'C02', 'C03', 'C04', 'C05', 'C08', 'C09', 'C10', 'C13', 'C14', 'C15', 'C19', 'C20', 'D01', 'D02', 'D03', 'D04', 'D05', 'D06', 'D07', 'D08', 'D09', 'D10', 'D11', 'D12', 'D13', 'D14', 'D15', 'D19', 'D22', 'D23', 'D24', 'D25', 'D26', 'D28', 'D29', 'D33', 'D42', 'D43', 'D44', 'D45', 'D63', 'D64', 'D66', 'D71', 'D72', 'D78', 'D80', 'D81', 'D82', 'D83', 'D84', 'D90', 'D91', 'D96', 'D97', 'H00', 'H01', 'H02', 'H03', 'H04', 'H05', 'H06', 'H07', 'H08', 'K01', 'M00', 'M01', 'M02', 'M03', 'M04', 'M05', 'M06', 'M07', 'M08', 'M09', 'M10', 'M11', 'M13', 'M14', 'N00', 'N01', 'N03', 'N05', 'R01', 'R02', 'R03', 'R04', 'S00', 'T00', 'T01', 'T02', 'T03', 'T04', 'T05', 'T06', 'T07', 'V00', 'V01', 'V02', 'V03', 'V04', 'V05', 'V06', 'V07', 'V08', 'V09', 'V10']


C:\Users\Admin\AppData\Local\Temp\ipykernel_3868\2512395822.py:14: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'has_{combo}'] = df['to_hop_mon'].apply(lambda x: 1 if combo in x else 0)
C:\Users\Admin\AppData\Local\Temp\ipykernel_3868\2512395822.py:14: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'has_{combo}'] = df['to_hop_mon'].apply(lambda x: 1 if combo in x else 0)
C:\Users\Admin\AppData\Local\Temp\ipykernel_3868\2512395822.py:14: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of ca

Fitting 5 folds for each of 72 candidates, totalling 360 fits

 Tham số tốt nhất: {'colsample_bytree': 0.7, 'learning_rate': 0.1, 'max_depth': 10, 'n_estimators': 200, 'subsample': 0.7}
 Đã lưu dự đoán năm 2024 vào 'thang_30/2024_thang_30.csv'.
 MAE: 3.743341015044843
 MSE: 21.25859429744262
 R^2: -0.16001133158386582

 Các ngành mới (newmajor == 1) trong năm 2024 và điểm dự đoán:
Empty DataFrame
Columns: [ma_nganh, ten_nganh, predicted_diem]
Index: []


C:\Users\Admin\AppData\Local\Temp\ipykernel_3868\2512395822.py:71: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  output_df['nam'] = test_year


In [241]:
predict_admission_scores('thang_30/thang_30.csv', 'thang_30/thang_30.txt', 'thang_30/2025_thang_30.csv', train_years=[2020, 2021, 2022, 2023, 2024])


 Danh sách tổ hợp môn: ['A00', 'A01', 'A02', 'A03', 'A04', 'A05', 'A06', 'A07', 'A08', 'A09', 'A10', 'A11', 'A12', 'A14', 'A15', 'A16', 'A17', 'B00', 'B01', 'B02', 'B03', 'B04', 'B05', 'B08', 'C00', 'C01', 'C02', 'C03', 'C04', 'C05', 'C08', 'C09', 'C10', 'C13', 'C14', 'C15', 'C19', 'C20', 'D01', 'D02', 'D03', 'D04', 'D05', 'D06', 'D07', 'D08', 'D09', 'D10', 'D11', 'D12', 'D13', 'D14', 'D15', 'D19', 'D22', 'D23', 'D24', 'D25', 'D26', 'D28', 'D29', 'D33', 'D42', 'D43', 'D44', 'D45', 'D63', 'D64', 'D66', 'D71', 'D72', 'D78', 'D80', 'D81', 'D82', 'D83', 'D84', 'D90', 'D91', 'D96', 'D97', 'H00', 'H01', 'H02', 'H03', 'H04', 'H05', 'H06', 'H07', 'H08', 'K01', 'M00', 'M01', 'M02', 'M03', 'M04', 'M05', 'M06', 'M07', 'M08', 'M09', 'M10', 'M11', 'M13', 'M14', 'N00', 'N01', 'N03', 'N05', 'R01', 'R02', 'R03', 'R04', 'S00', 'T00', 'T01', 'T02', 'T03', 'T04', 'T05', 'T06', 'T07', 'V00', 'V01', 'V02', 'V03', 'V04', 'V05', 'V06', 'V07', 'V08', 'V09', 'V10']


C:\Users\Admin\AppData\Local\Temp\ipykernel_3868\2512395822.py:14: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'has_{combo}'] = df['to_hop_mon'].apply(lambda x: 1 if combo in x else 0)
C:\Users\Admin\AppData\Local\Temp\ipykernel_3868\2512395822.py:14: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'has_{combo}'] = df['to_hop_mon'].apply(lambda x: 1 if combo in x else 0)
C:\Users\Admin\AppData\Local\Temp\ipykernel_3868\2512395822.py:14: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of ca

Fitting 5 folds for each of 72 candidates, totalling 360 fits

 Tham số tốt nhất: {'colsample_bytree': 1.0, 'learning_rate': 0.2, 'max_depth': 10, 'n_estimators': 200, 'subsample': 1.0}
 Đã lưu dự đoán năm 2025 vào 'thang_30/2025_thang_30.csv'.
 Không có dữ liệu thật cho năm 2025, không tính được các metric.

 Các ngành mới (newmajor == 1) trong năm 2025 và điểm dự đoán:
     ma_nganh                                                                                                                                                                     ten_nganh  predicted_diem
          MS5                                                                                                                                                                   Kỹ thuật in       23.370001
          EM1                                                                                                                                                            Quản lý năng lượng       24.840000
          ED3 

C:\Users\Admin\AppData\Local\Temp\ipykernel_3868\2512395822.py:71: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  output_df['nam'] = test_year


#OUTPUT

In [242]:
file_paths = [
    './qsb/2025_qsb.csv',
    './thang_30/2025_thang_30.csv',
    './thang_40/2025_thang_40.csv'
]

dfs = []
for path in file_paths:
    df = pd.read_csv(path)
    dfs.append(df)

merged_df = pd.concat(dfs, ignore_index=True)
merged_df = merged_df.rename(columns={'predicted_diem': 'diem'})
output_df = merged_df[['ma_truong', 'ten_truong', 'dia_diem', 'ma_nganh', 'ten_nganh', 'diem', 'to_hop_mon']]
output_df.to_csv('./data/predict_2025.csv', index=False)
print(" Đã lưu file kết quả tại './data/predict_2025.csv'.")


 Đã lưu file kết quả tại './data/predict_2025.csv'.
